In [68]:
# 📦 Importação dos pacotes
import pandas as pd
import sqlite3
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# 🔌 Conexão com o banco de dados
conn = sqlite3.connect("Data/G2.db")

# 📥 Carregando as tabelas
casos_municipais = pd.read_sql_query("SELECT * FROM CasosMunicipais", conn)
casos_padronizados = pd.read_sql_query("SELECT * FROM CasosPadronizados", conn)
municipios = pd.read_sql_query("SELECT * FROM Municipio", conn)
estados = pd.read_sql_query("SELECT * FROM Estado", conn)
regioes = pd.read_sql_query("SELECT * FROM Regiao", conn)

# ❌ Encerrando conexão
conn.close()

# 🔗 Realizando JOINs para integrar todos os dados
casos = casos_municipais.merge(municipios, on='codigomunicipal', how='left') \
                        .merge(estados, on='codigoestadual', how='left') \
                        .merge(regioes, on='codigoregiao', how='left')

# 🧹 Convertendo datas para datetime
casos['data'] = pd.to_datetime(casos['data'], dayfirst=True)

# 🔧 Preenchendo valores nulos com 0
casos['casosnovos'] = casos['casosnovos'].fillna(0)
casos['mortesnovas'] = casos['mortesnovas'].fillna(0)

# 🔢 Agrupamento por município, estado e data para cálculo de totais diários
casos_agg = casos.groupby([
    'data', 'codigomunicipal', 'nomecidade', 'uf', 'nomeestado', 'nomeregiao', 'populacao'
]).agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum'
}).reset_index()

# 🧮 Cálculo de acumulados ao longo do tempo por município
casos_agg['casosacumulado'] = casos_agg.groupby('codigomunicipal')['casosnovos'].cumsum()
casos_agg['mortesacumulado'] = casos_agg.groupby('codigomunicipal')['mortesnovas'].cumsum()


In [69]:
# 🟦 Indicadores do Brasil
brasil = casos.groupby('data').agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum',
    'populacao': 'sum'
}).reset_index()

brasil_totais = brasil[['casosnovos', 'mortesnovas']].sum()
pop_brasil = brasil['populacao'].max()
letalidade_brasil = brasil_totais['mortesnovas'] / brasil_totais['casosnovos']
mortalidade_brasil = brasil_totais['mortesnovas'] / pop_brasil

# 🟥 Indicadores do Estado do Rio de Janeiro (ERJ)
casos_rj = casos[casos['uf'] == 'RJ']
rj_totais = casos_rj.groupby('data').agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum'
}).sum()
pop_rj = casos_rj['populacao'].dropna().unique().sum()
letalidade_rj = rj_totais['mortesnovas'] / rj_totais['casosnovos']
mortalidade_rj = rj_totais['mortesnovas'] / pop_rj

# 🟨 Indicadores do Município do Rio de Janeiro (MRJ)
casos_mrj = casos[casos['nomecidade'].str.upper() == 'RIO DE JANEIRO']
mrj_totais = casos_mrj.groupby('data').agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum'
}).sum()
pop_mrj = casos_mrj['populacao'].dropna().unique().sum()
letalidade_mrj = mrj_totais['mortesnovas'] / mrj_totais['casosnovos']
mortalidade_mrj = mrj_totais['mortesnovas'] / pop_mrj


In [70]:
# 🧮 Casos acumulados por estado
casos_estado = casos.groupby('uf').agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum',
    'populacao': 'max'  # População estadual já está agregada no município com maior valor
}).reset_index()

# 🧾 Cálculo de letalidade e mortalidade por estado
casos_estado['letalidade'] = casos_estado['mortesnovas'] / casos_estado['casosnovos']
casos_estado['mortalidade'] = casos_estado['mortesnovas'] / casos_estado['populacao']

# 🧽 Lidando com divisão por zero (casos = 0)
casos_estado.replace([np.inf, -np.inf], np.nan, inplace=True)
casos_estado.fillna(0, inplace=True)

# 📋 Renomeando colunas para exibição
casos_estado.rename(columns={
    'uf': 'UF',
    'casosnovos': 'Confirmados',
    'mortesnovas': 'Obitos',
    'populacao': 'Populacao'
}, inplace=True)

# 📊 Ordenando por UF
casos_estado.sort_values(by='UF', inplace=True)


In [72]:
# 🎯 Agrupamento por região
casos_regiao = casos.groupby(['nomeregiao']).agg({
    'casosnovos': 'sum',
    'populacao': 'sum'
}).reset_index()

# 📊 Cálculo de taxa por 100 mil habitantes
casos_regiao['taxa_casos_100k'] = (casos_regiao['casosnovos'] / casos_regiao['populacao']) * 100000

# 📋 Renomeando para exibição
casos_regiao.rename(columns={
    'nomeregiao': 'Regiao',
    'casosnovos': 'Confirmados'
}, inplace=True)

# 🔽 Ordenar por taxa (opcional)
casos_regiao.sort_values(by='taxa_casos_100k', ascending=False, inplace=True)


In [73]:
# 🔎 Filtrando apenas Sudeste
sudeste = casos[casos['nomeregiao'].str.upper() == 'SUDESTE']

# 🧮 Agrupamento por estado
casos_sudeste = sudeste.groupby('uf').agg({
    'casosnovos': 'sum',
    'populacao': 'sum'
}).reset_index()

# 📊 Taxa por 100 mil habitantes
casos_sudeste['taxa_casos_100k'] = (casos_sudeste['casosnovos'] / casos_sudeste['populacao']) * 100000

# 📋 Renomeando colunas
casos_sudeste.rename(columns={
    'uf': 'UF',
    'casosnovos': 'Confirmados'
}, inplace=True)

# 🔽 Ordenar por taxa (opcional)
casos_sudeste.sort_values(by='taxa_casos_100k', ascending=False, inplace=True)


In [74]:
# 🎯 Agregando por cidade
cidades = casos[casos['nomecidade'].notna()]
cidades_agg = cidades.groupby(['uf', 'nomecidade', 'populacao']).agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum'
}).reset_index()

# 📊 Taxas
cidades_agg['taxa_casos_100k'] = (cidades_agg['casosnovos'] / cidades_agg['populacao']) * 100000
cidades_agg['letalidade'] = cidades_agg['mortesnovas'] / cidades_agg['casosnovos']
cidades_agg['mortalidade'] = cidades_agg['mortesnovas'] / cidades_agg['populacao']

# 🏙️ Top 10 BRASIL
top10_brasil = cidades_agg.sort_values(by='taxa_casos_100k', ascending=False).head(10)
top10_brasil.rename(columns={'nomecidade': 'Cidade', 'uf': 'UF'}, inplace=True)

# 🏙️ Top 10 RJ
top10_rj = cidades_agg[cidades_agg['uf'] == 'RJ'].sort_values(by='taxa_casos_100k', ascending=False).head(10)
top10_rj.rename(columns={'nomecidade': 'Cidade'}, inplace=True)


In [75]:
# 🔹 Brasil: agrupando casos novos por data
casos_brasil_dia = casos[casos['nomecidade'].notna()].groupby('data').agg({
    'casosnovos': 'sum'
}).rename(columns={'casosnovos': 'Brasil'}).reset_index()

# 🔹 RJ: filtrando e agrupando por data
casos_rj_dia = casos[(casos['uf'] == 'RJ') & (casos['nomecidade'].notna())].groupby('data').agg({
    'casosnovos': 'sum'
}).rename(columns={'casosnovos': 'RJ'}).reset_index()

# 🔄 Merge entre as duas séries de tempo
casos_dia = pd.merge(casos_brasil_dia, casos_rj_dia, on='data', how='outer').fillna(0)

# ➕ Adiciona contador de dias para eixo X
casos_dia['dia'] = range(1, len(casos_dia) + 1)


In [76]:
# 🔹 Brasil: média móvel de 6 dias
mortes_brasil_dia = casos[casos['nomecidade'].notna()].groupby('data').agg({
    'mortesnovas': 'sum'
}).rename(columns={'mortesnovas': 'Brasil'}).reset_index()

# 🔹 RJ
mortes_rj_dia = casos[(casos['uf'] == 'RJ') & (casos['nomecidade'].notna())].groupby('data').agg({
    'mortesnovas': 'sum'
}).rename(columns={'mortesnovas': 'RJ'}).reset_index()

# 🔄 Merge e cálculo da média móvel
mortes_dia = pd.merge(mortes_brasil_dia, mortes_rj_dia, on='data', how='outer').fillna(0)
mortes_dia['dia'] = range(1, len(mortes_dia) + 1)
mortes_dia['mm_brasil'] = mortes_dia['Brasil'].rolling(window=6, min_periods=1).mean()
mortes_dia['mm_rj'] = mortes_dia['RJ'].rolling(window=6, min_periods=1).mean()

# 🔍 Tendência: compara último valor com valor 14 dias antes
def analisar_tendencia(serie):
    if len(serie) < 15:
        return "Insuficiente"
    atual = serie.iloc[-1]
    anterior = serie.iloc[-15]
    variacao = (atual - anterior) / anterior if anterior != 0 else 0
    if abs(variacao) <= 0.15:
        return "Estável"
    elif variacao > 0.15:
        return "Em crescimento"
    else:
        return "Em queda"

tendencia_brasil = analisar_tendencia(mortes_dia['mm_brasil'])
tendencia_rj = analisar_tendencia(mortes_dia['mm_rj'])


In [77]:
# 🔄 RJ: agregação por data
rj_acumulado = casos[casos['uf'] == 'RJ'].groupby('data').agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum'
}).reset_index()

# 📈 Cálculo de acumulados
rj_acumulado['casosacumulado'] = rj_acumulado['casosnovos'].cumsum()
rj_acumulado['mortesacumulado'] = rj_acumulado['mortesnovas'].cumsum()
rj_acumulado['dia'] = range(1, len(rj_acumulado) + 1)


In [82]:
# 📦 Importações
import pandas as pd
import sqlite3
import numpy as np
import geopandas as gpd
import json
import plotly.express as px
from dash import Dash, html, dcc
# 🗺️ Carregar shapefiles e transformar em GeoJSON
shp_brasil = gpd.read_file("Mapas/BR_UF_2022.shp")
shp_brasil = shp_brasil.rename(columns={"CD_UF": "codigoibge"})
geojson_brasil = json.loads(shp_brasil.to_json())

shp_rj = gpd.read_file("Mapas/RJ_Municipios_2022.shp")
shp_rj = shp_rj.rename(columns={"CD_MUN": "codigomunicipal"})
geojson_rj = json.loads(shp_rj.to_json())

# 🔷 Mapas
mapa_brasil = px.choropleth(
    casos_estado.merge(shp_brasil, left_on="UF", right_on="SIGLA_UF"),
    geojson=geojson_brasil,
    locations="codigoibge",
    color="Confirmados",
    featureidkey="properties.codigoibge",
    title="Casos Confirmados por Estado – Brasil"
)

mapa_rj = px.choropleth(
    casos[casos['uf'] == 'RJ'].groupby('codigomunicipal').agg({'casosnovos': 'sum'}).reset_index().merge(shp_rj, on='codigomunicipal'),
    geojson=geojson_rj,
    locations="codigomunicipal",
    color="casosnovos",
    featureidkey="properties.codigomunicipal",
    title="Casos Confirmados por Município – RJ"
)

# 🔶 Gráficos de barras
bar_regiao = px.bar(casos_regiao, x="Regiao", y="taxa_casos_100k", title="Taxa de Casos por Região")
bar_sudeste = px.bar(casos_sudeste, x="UF", y="taxa_casos_100k", title="Taxa de Casos – Sudeste")
bar_top10_br = px.bar(top10_brasil, x="Cidade", y="taxa_casos_100k", title="Top 10 Cidades – Brasil")
bar_top10_rj = px.bar(top10_rj, x="Cidade", y="taxa_casos_100k", title="Top 10 Cidades – RJ")

# 🔷 Gráficos temporais
linha_casos = px.line(casos_dia, x="dia", y=["Brasil", "RJ"], title="Casos Novos por Dia")
linha_mortes = px.line(mortes_dia, x="dia", y=["mm_brasil", "mm_rj"], title="Média Móvel de Mortes")
linha_rj = px.line(rj_acumulado, x="dia", y=["casosacumulado", "mortesacumulado"], title="Casos e Mortes Acumuladas – RJ")

# 🧾 Tabela por estado
tabela_estado = dcc.Graph(
    figure=px.bar(casos_estado, x="UF", y="Confirmados", title="Casos Confirmados por Estado (Barra)")
)

# 🖼️ Layout do dashboard
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Painel Epidemiológico – G2"),

    html.Div([
        html.Div([
            html.H3("Mapa – Casos por Estado"),
            dcc.Graph(figure=mapa_brasil)
        ], style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            html.H3("Mapa – Casos por Município (RJ)"),
            dcc.Graph(figure=mapa_rj)
        ], style={'width': '49%', 'display': 'inline-block'}),
    ]),

    html.Div([
        dcc.Graph(figure=bar_regiao),
        dcc.Graph(figure=bar_sudeste),
    ], style={'columnCount': 2}),

    html.Div([
        dcc.Graph(figure=bar_top10_br),
        dcc.Graph(figure=bar_top10_rj),
    ], style={'columnCount': 2}),

    html.Div([
        dcc.Graph(figure=linha_casos),
        dcc.Graph(figure=linha_mortes),
        html.Div(f"Tendência BR: {tendencia_brasil} | RJ: {tendencia_rj}", style={'textAlign': 'center'})
    ]),

    html.Div([
        dcc.Graph(figure=linha_rj)
    ]),

    html.Div([
        html.H3("Tabela por Estado"),
        tabela_estado
    ])
])

if __name__ == "__main__":
    app.run(debug=True)

In [2]:
# app.py
import pandas as pd
import sqlite3
import numpy as np
import geopandas as gpd
import json
import plotly.express as px
from dash import Dash, html, dcc

# === DADOS ===
conn = sqlite3.connect("Data/G2.db")
casos_municipais = pd.read_sql("SELECT * FROM CasosMunicipais", conn)
municipios = pd.read_sql("SELECT * FROM Municipio", conn)
estados = pd.read_sql("SELECT * FROM Estado", conn)
regioes = pd.read_sql("SELECT * FROM Regiao", conn)
conn.close()

casos = casos_municipais.merge(municipios, on='codigomunicipal') \
                        .merge(estados, on='codigoestadual') \
                        .merge(regioes, on='codigoregiao')

casos['data'] = pd.to_datetime(casos['data'], dayfirst=True)
casos['casosnovos'] = casos['casosnovos'].fillna(0)
casos['mortesnovas'] = casos['mortesnovas'].fillna(0)

# === Cálculo dos totais ===
brasil = casos.groupby('data').agg({'casosnovos': 'sum', 'mortesnovas': 'sum', 'populacao': 'sum'}).reset_index()
brasil_totais = brasil[['casosnovos', 'mortesnovas']].sum()
pop_brasil = brasil['populacao'].max()
letalidade_brasil = brasil_totais['mortesnovas'] / brasil_totais['casosnovos']
mortalidade_brasil = brasil_totais['mortesnovas'] / pop_brasil

casos_rj = casos[casos['uf'] == 'RJ']
rj_totais = casos_rj.groupby('data').agg({'casosnovos': 'sum', 'mortesnovas': 'sum'}).sum()
pop_rj = casos_rj['populacao'].dropna().unique().sum()
letalidade_rj = rj_totais['mortesnovas'] / rj_totais['casosnovos']
mortalidade_rj = rj_totais['mortesnovas'] / pop_rj

casos_mrj = casos[casos['nomecidade'].str.upper() == 'RIO DE JANEIRO']
mrj_totais = casos_mrj.groupby('data').agg({'casosnovos': 'sum', 'mortesnovas': 'sum'}).sum()
pop_mrj = casos_mrj['populacao'].dropna().unique().sum()
letalidade_mrj = mrj_totais['mortesnovas'] / mrj_totais['casosnovos']
mortalidade_mrj = mrj_totais['mortesnovas'] / pop_mrj

# === Casos por estado ===
casos_estado = casos.groupby('uf').agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum',
    'populacao': 'max'
}).reset_index()
casos_estado['letalidade'] = casos_estado['mortesnovas'] / casos_estado['casosnovos']
casos_estado['mortalidade'] = casos_estado['mortesnovas'] / casos_estado['populacao']
casos_estado.rename(columns={'uf': 'UF', 'casosnovos': 'Confirmados', 'mortesnovas': 'Obitos', 'populacao': 'Populacao'}, inplace=True)

# === Gráfico por estado ===
grafico_estado = px.bar(casos_estado, x="UF", y="Confirmados", title="Casos Confirmados por Estado")

# === Gráfico por região ===
casos_regiao = casos.groupby('nomeregiao').agg({
    'casosnovos': 'sum',
    'populacao': 'sum'
}).reset_index()
casos_regiao['taxa_casos_100k'] = (casos_regiao['casosnovos'] / casos_regiao['populacao']) * 100000
bar_regiao = px.bar(casos_regiao, x="nomeregiao", y="taxa_casos_100k", title="Taxa de Casos por Região")

# === Gráfico Sudeste ===
sudeste = casos[casos['nomeregiao'].str.upper() == 'SUDESTE']
casos_sudeste = sudeste.groupby('uf').agg({'casosnovos': 'sum', 'populacao': 'sum'}).reset_index()
casos_sudeste['taxa_casos_100k'] = (casos_sudeste['casosnovos'] / casos_sudeste['populacao']) * 100000
bar_sudeste = px.bar(casos_sudeste, x='uf', y='taxa_casos_100k', title='Taxa de Casos – Sudeste')

# === Top 10 cidades do Brasil e RJ ===
cidades = casos[casos['nomecidade'].notna()]
cidades_agg = cidades.groupby(['uf', 'nomecidade', 'populacao']).agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum'
}).reset_index()
cidades_agg['taxa'] = cidades_agg['casosnovos'] / cidades_agg['populacao']
cidades_agg['letalidade'] = cidades_agg['mortesnovas'] / cidades_agg['casosnovos']
cidades_agg['mortalidade'] = cidades_agg['mortesnovas'] / cidades_agg['populacao']
top10_brasil = cidades_agg.sort_values(by='taxa', ascending=False).head(10)
top10_rj = cidades_agg[cidades_agg['uf'] == 'RJ'].sort_values(by='taxa', ascending=False).head(10)
bar_top10_br = px.bar(top10_brasil, x="nomecidade", y="taxa", title="Top 10 Cidades – Brasil")
bar_top10_rj = px.bar(top10_rj, x="nomecidade", y="taxa", title="Top 10 Cidades – RJ")

# === Casos por dia ===
casos_dia = casos.groupby(['data', 'uf']).agg({'casosnovos': 'sum'}).reset_index()
brasil_dia = casos_dia.groupby('data')['casosnovos'].sum().reset_index(name='Brasil')
rj_dia = casos_dia[casos_dia['uf'] == 'RJ'][['data', 'casosnovos']].rename(columns={'casosnovos': 'RJ'})
casos_dia = pd.merge(brasil_dia, rj_dia, on='data', how='outer').fillna(0)
casos_dia['dia'] = range(1, len(casos_dia) + 1)
linha_casos = px.line(casos_dia, x="dia", y=["Brasil", "RJ"], title="Casos Novos por Dia")

# === Média móvel de óbitos ===
mortes_brasil = casos.groupby('data')['mortesnovas'].sum().rolling(6).mean().reset_index(name='Brasil')
mortes_rj = casos[casos['uf'] == 'RJ'].groupby('data')['mortesnovas'].sum().rolling(6).mean().reset_index(name='RJ')
mortes_dia = pd.merge(mortes_brasil, mortes_rj, on='data', how='outer').fillna(0)
mortes_dia['dia'] = range(1, len(mortes_dia) + 1)
linha_mortes = px.line(mortes_dia, x="dia", y=["Brasil", "RJ"], title="Média Móvel de Óbitos")

# === RJ acumulado ===
rj_acumulado = casos[casos['uf'] == 'RJ'].groupby('data').agg({'casosnovos': 'sum', 'mortesnovas': 'sum'}).cumsum().reset_index()
rj_acumulado['dia'] = range(1, len(rj_acumulado) + 1)
linha_rj = px.line(rj_acumulado, x="dia", y=["casosnovos", "mortesnovas"], title="Casos e Óbitos Acumulados – RJ")

# === Layout final ===
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Painel Epidemiológico – G2"),

    html.Div([
        html.Div([html.H4("Casos"), html.P(f"BR: {int(brasil_totais['casosnovos'])}"),
                  html.P(f"RJ: {int(rj_totais['casosnovos'])}"), html.P(f"MRJ: {int(mrj_totais['casosnovos'])}")],
                 style={'width': '24%', 'display': 'inline-block'}),
        html.Div([html.H4("Óbitos"), html.P(f"BR: {int(brasil_totais['mortesnovas'])}"),
                  html.P(f"RJ: {int(rj_totais['mortesnovas'])}"), html.P(f"MRJ: {int(mrj_totais['mortesnovas'])}")],
                 style={'width': '24%', 'display': 'inline-block'}),
        html.Div([html.H4("Letalidade"), html.P(f"BR: {letalidade_brasil:.2%}"),
                  html.P(f"RJ: {letalidade_rj:.2%}"), html.P(f"MRJ: {letalidade_mrj:.2%}")],
                 style={'width': '24%', 'display': 'inline-block'}),
        html.Div([html.H4("Mortalidade"), html.P(f"BR: {mortalidade_brasil:.2%}"),
                  html.P(f"RJ: {mortalidade_rj:.2%}"), html.P(f"MRJ: {mortalidade_mrj:.2%}")],
                 style={'width': '24%', 'display': 'inline-block'})
    ]),

    html.Div([
        dcc.Graph(figure=linha_casos),
        dcc.Graph(figure=linha_mortes),
        dcc.Graph(figure=linha_rj)
    ]),

    html.Div([
        dcc.Graph(figure=grafico_estado),
        dcc.Graph(figure=bar_regiao),
        dcc.Graph(figure=bar_sudeste),
        dcc.Graph(figure=bar_top10_br),
        dcc.Graph(figure=bar_top10_rj)
    ])
])

if __name__ == '__main__':
    app.run(debug=True)


In [4]:
# app.py
import pandas as pd
import sqlite3
import numpy as np
import geopandas as gpd
import json
import plotly.express as px
from dash import Dash, html, dcc

# === DADOS ===
conn = sqlite3.connect("Data/G2.db")
casos_municipais = pd.read_sql("SELECT * FROM CasosMunicipais", conn)
municipios = pd.read_sql("SELECT * FROM Municipio", conn)
estados = pd.read_sql("SELECT * FROM Estado", conn)
regioes = pd.read_sql("SELECT * FROM Regiao", conn)
conn.close()

casos = casos_municipais.merge(municipios, on='codigomunicipal') \
                        .merge(estados, on='codigoestadual') \
                        .merge(regioes, on='codigoregiao')

casos['data'] = pd.to_datetime(casos['data'], dayfirst=True)
casos['casosnovos'] = casos['casosnovos'].fillna(0)
casos['mortesnovas'] = casos['mortesnovas'].fillna(0)

# === Cálculo dos totais ===
brasil = casos.groupby('data').agg({'casosnovos': 'sum', 'mortesnovas': 'sum', 'populacao': 'sum'}).reset_index()
brasil_totais = brasil[['casosnovos', 'mortesnovas']].sum()
pop_brasil = brasil['populacao'].max()
letalidade_brasil = brasil_totais['mortesnovas'] / brasil_totais['casosnovos']
mortalidade_brasil = brasil_totais['mortesnovas'] / pop_brasil

casos_rj = casos[casos['uf'] == 'RJ']
rj_totais = casos_rj.groupby('data').agg({'casosnovos': 'sum', 'mortesnovas': 'sum'}).sum()
pop_rj = casos_rj['populacao'].dropna().unique().sum()
letalidade_rj = rj_totais['mortesnovas'] / rj_totais['casosnovos']
mortalidade_rj = rj_totais['mortesnovas'] / pop_rj

casos_mrj = casos[casos['nomecidade'].str.upper() == 'RIO DE JANEIRO']
mrj_totais = casos_mrj.groupby('data').agg({'casosnovos': 'sum', 'mortesnovas': 'sum'}).sum()
pop_mrj = casos_mrj['populacao'].dropna().unique().sum()
letalidade_mrj = mrj_totais['mortesnovas'] / mrj_totais['casosnovos']
mortalidade_mrj = mrj_totais['mortesnovas'] / pop_mrj

# === Casos por estado ===
casos_estado = casos.groupby('uf').agg({
    'casosnovos': 'sum',
    'mortesnovas': 'sum',
    'populacao': 'max'
}).reset_index()
casos_estado['letalidade'] = casos_estado['mortesnovas'] / casos_estado['casosnovos']
casos_estado['mortalidade'] = casos_estado['mortesnovas'] / casos_estado['populacao']
casos_estado.rename(columns={'uf': 'UF', 'casosnovos': 'Confirmados', 'mortesnovas': 'Obitos', 'populacao': 'Populacao'}, inplace=True)

# === Shapefiles e GeoJSON ===
shp_brasil = gpd.read_file("Mapas/BR_UF_2022.shp").rename(columns={"CD_UF": "codigoibge"})
shp_rj = gpd.read_file("Mapas/RJ_Municipios_2022.shp").rename(columns={"CD_MUN": "codigomunicipal"})
geojson_brasil = json.loads(shp_brasil.to_json())
geojson_rj = json.loads(shp_rj.to_json())

casos_estado = casos_estado.merge(shp_brasil, left_on="UF", right_on="SIGLA_UF")
mapa_brasil = px.choropleth(
    casos_estado,
    geojson=geojson_brasil,
    locations="codigoibge",
    color="Confirmados",
    featureidkey="properties.codigoibge",
    title="Casos Confirmados por Estado – Brasil"
)
mapa_brasil.update_geos(fitbounds="locations", visible=False)

dados_mapa_rj = casos[casos['uf'] == 'RJ'].groupby('codigomunicipal').agg({'casosnovos': 'sum'}).reset_index()
dados_mapa_rj = dados_mapa_rj.merge(shp_rj, on='codigomunicipal')
mapa_rj = px.choropleth(
    dados_mapa_rj,
    geojson=geojson_rj,
    locations='codigomunicipal',
    color='casosnovos',
    featureidkey='properties.codigomunicipal',
    title='Casos Confirmados por Município – RJ'
)
mapa_rj.update_geos(fitbounds="locations", visible=False)

# === Dash app ===
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Painel Epidemiológico – G2"),

    html.Div([
        html.Div([dcc.Graph(figure=mapa_brasil)], style={'width': '49%', 'display': 'inline-block'}),
        html.Div([dcc.Graph(figure=mapa_rj)], style={'width': '49%', 'display': 'inline-block'})
    ])
])

if __name__ == '__main__':
    app.run(debug=True)


In [3]:
# template_dashboard.py
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

# Inicializa o app
app = Dash(__name__)

# Exemplo de dados
df = pd.DataFrame({
    'data': pd.date_range(start='2020-01-01', periods=10, freq='D'),
    'casos': [10, 20, 30, 25, 40, 50, 45, 60, 70, 80],
    'estado': ['RJ']*5 + ['SP']*5,
    'confirmados': [100, 200, 150, 250, 300, 120, 130, 210, 280, 350]
})

# Gráficos de exemplo
fig_linha = px.line(df, x='data', y='casos', title='Casos ao longo do tempo')
fig_barra = px.bar(df, x='estado', y='confirmados', title='Casos por estado')

# Layout com Tabs
app.layout = html.Div([
    html.H1("Painel Interativo com Tabs"),

    dcc.Tabs([
        
        dcc.Tab(label='Visão Geral', children=[
            html.Div([
                html.H3("Indicadores – Visão Geral"),

                html.Div([
                    html.Div([
                        html.H4("Casos Confirmados"),
                        html.P(f"Brasil: {int(brasil_totais['casosnovos']):,}".replace(',', '.')),
                        html.P(f"ERJ: {int(rj_totais['casosnovos']):,}".replace(',', '.')),
                        html.P(f"MRJ: {int(mrj_totais['casosnovos']):,}".replace(',', '.'))
                    ], style={'width': '24%', 'display': 'inline-block', 'padding': '10px'}),

                    html.Div([
                        html.H4("Óbitos Confirmados"),
                        html.P(f"Brasil: {int(brasil_totais['mortesnovas']):,}".replace(',', '.')),
                        html.P(f"ERJ: {int(rj_totais['mortesnovas']):,}".replace(',', '.')),
                        html.P(f"MRJ: {int(mrj_totais['mortesnovas']):,}".replace(',', '.'))
                    ], style={'width': '24%', 'display': 'inline-block', 'padding': '10px'}),

                    html.Div([
                        html.H4("Letalidade"),
                        html.P(f"Brasil: {letalidade_brasil:.2%}"),
                        html.P(f"ERJ: {letalidade_rj:.2%}"),
                        html.P(f"MRJ: {letalidade_mrj:.2%}")
                    ], style={'width': '24%', 'display': 'inline-block', 'padding': '10px'}),

                    html.Div([
                        html.H4("Mortalidade"),
                        html.P(f"Brasil: {mortalidade_brasil * 1000:.2f}‰"),
                        html.P(f"ERJ: {mortalidade_rj * 1000:.2f}‰"),
                        html.P(f"MRJ: {mortalidade_mrj * 1000:.2f}‰")
                    ], style={'width': '24%', 'display': 'inline-block', 'padding': '10px'}),
                ])
            ])
        ]),

        dcc.Tab(label='Gráficos', children=[
            html.Div([
                html.Div([
                    dcc.Graph(figure=fig_linha)
                ], style={'width': '49%', 'display': 'inline-block'}),

                html.Div([
                    dcc.Graph(figure=fig_barra)
                ], style={'width': '49%', 'display': 'inline-block'})
            ])
        ]),

        dcc.Tab(label='Tabela', children=[
            html.Div([
                html.H4("Tabela de Dados"),
                dcc.Graph(figure=px.scatter(df, x='data', y='casos', color='estado'))
            ])
        ])
    ])
], style={'backgroundColor': 'white', 'padding': '20px'})

# Executa o servidor
if __name__ == '__main__':
    app.run(debug=True)
